# Callbacks
> Callbacks are one of the most amazing functionality in fastai, they're the most essential pillar for fastai tremendous modularity.

## Required imports

In [ ]:
from fastai2.basics import *
from fastai2.callback.all import *

In [ ]:
from fastcook.utils import *

## Available callbacks

There is a callback for **every** step of the training loop.   
Check all the available callbacks [here](http://dev.fast.ai/callback.core#Callback) or check their definitions in the source code by running the cells below:

In [ ]:
Learner.one_batch??

In [ ]:
Learner.fit??

## How to use?

To create a callback you simply need to inherit from `Callback` and define methods with the event names you want to interact with. 


In [ ]:
class PrintCallback(Callback):
  def after_epoch(self): print('After epoch')
  def begin_fit(self):   print('Beginning fit')

In [ ]:
learn = synth_learner(cbs=PrintCallback())

In [ ]:
learn.fit(2)

Beginning fit


epoch,train_loss,valid_loss,time
0,10.417525,9.759871,00:00
1,10.357862,9.689531,00:00


After epoch
After epoch


Now, the **really** cool thing about callbacks is that they have **access to the learner** object itself. In the `Learner` training loop **everything** ends up being saved as an attribute, the predictions, loss, targets, everything. This gives the callback complete power to modify anything you need.

Let's define a custom loss function that receives the standard combination of predictions and targets *plus* some additional stuff.

In [ ]:
class ExplodingCallback(Callback):
  def after_pred(self):
    state = 'stable'
    if random.randint(0,1): state = 'explode'
    self.learn.yb = (*self.yb, state)

**Note:** You need to use `self.learn.<stuff>` to write stuff but only `self.<stuff>` to read it.

In [ ]:
def explosive_loss(pred, targ, state, **kwargs):
  loss = MSELossFlat()(pred,targ,**kwargs)
  return loss + (1000 if state=='explode' else 0)

In [ ]:
learn = synth_learner(cbs=ExplodingCallback(), loss_func=explosive_loss)
learn.fit(1)

epoch,train_loss,valid_loss,time
0,303.497498,503.368195,00:00


Let's also create a callback that stops training if explosions happen.

In [ ]:
class DefuserCallback(Callback):
  def after_loss(self):
    if self.loss > 1000:
      print('The bomb has been defused')
      raise CancelFitException

In [ ]:
cbs = [ExplodingCallback(), DefuserCallback()]
learn = synth_learner(cbs=cbs, loss_func=explosive_loss)
learn.fit(1)

The bomb has been defused
